## Dosage Form by Active

This is an attempt to prepare and manipulate the dataset from FDA (orange book) in order to gather the dosage form of each molecule. We first import the libraries:

In [1]:
import pandas as pd
import numpy as np
import re

We import the dataset manually retrieved from:
https://www.fda.gov/drugs/drug-approvals-and-databases/orange-book-data-files
The delimiter in the dataset is a tilde (~)

In [2]:
orange_book = pd.read_csv("../data/RAW_datasets/RAW_fda_orange_book.txt", delimiter = "~")

Taking a closer look in the dataset and the definitions below:

- Ingredient: The active ingredient(s) for the product. Multiple ingredients are in alphabetical order, separated by a semicolon.
- Dosage form; Route of Administration: The product dosage form and route separated by a semi-colon.  The format is not all uppercase.
- Trade Name: The trade name of the product as shown on the labeling.
- Applicant: The firm name holding legal responsibility for the new drug application.  The firm name is condensed to a maximum twenty character unique string.
- Strength: The potency of the active ingredient.  May repeat for multiple part products.
- New Drug Application Type: The type of new drug application approval.  New Drug Applications (NDA or innovator)  are ”N”. Abbreviated New Drug Applications (ANDA or generic) are “A”.
- New Drug Application (NDA): Number The FDA assigned number to the application. Format is nnnnnn. 
- Product Number: The FDA assigned number to identify the application products. Each strength is a separate product.  May repeat for multiple part products. Format is nnn.
- Therapeutic Equivalence (TE) Code: The TE Code indicates the therapeutic equivalence rating of generic to innovator Rx products.
- Approval Date: The date the product was approved as stated in the FDA approval letter to the applicant.  The format is Mmm dd, yyyy.  Products approved prior to the January 1, 1982 contain the phrase: "Approved prior to Jan 1, 1982".
- Reference Listed Drug (RLD): The RLD is a drug product approved under section 505(c) of the FD&C Act for which FDA has made a finding of safety and effectiveness. In the electronic Orange Book, an RLD is identified by “RLD” in the RLD column.
- Reference Standard (RS): A “reference standard” is the drug product selected by FDA that an applicant seeking approval of an ANDA must use in conducting an in vivo bioequivalence study required for approval of an ANDA.  In the electronic Orange Book, a reference standard is identified by “RS” in the RS column.
- Type: The group or category of approved drugs.  Format is RX, OTC, DISCN.
- Applicant Full Name: The full name of the firm holding legal responsibility for the new drug application. 


We take a first look on the database:

In [3]:
orange_book.head(6)

,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name
0,BUDESONIDE,"AEROSOL, FOAM;RECTAL",UCERIS,SALIX,2MG/ACTUATION,N,205613,1,NaN,"Oct 7, 2014",Yes,Yes,RX,SALIX PHARMACEUTICALS INC
1,MINOCYCLINE HYDROCHLORIDE,"AEROSOL, FOAM;TOPICAL",AMZEEQ,JOURNEY,EQ 4% BASE,N,212379,1,NaN,"Oct 18, 2019",Yes,Yes,RX,JOURNEY MEDICAL CORP
2,AZELAIC ACID,"AEROSOL, FOAM;TOPICAL",AZELAIC ACID,TEVA PHARMS USA,15%,A,210928,1,NaN,"Oct 7, 2020",No,No,DISCN,TEVA PHARMACEUTICALS USA INC
3,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,PADAGIS ISRAEL,0.12%,A,78337,1,AB,"Nov 26, 2012",No,No,RX,PADAGIS ISRAEL PHARMACEUTICALS LTD
4,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,NaN,"May 24, 2017",No,No,DISCN,RICONPHARMA LLC
5,BETAMETHASONE VALERATE,"AEROSOL, FOAM;TOPICAL",BETAMETHASONE VALERATE,TARO,0.12%,A,208204,1,AB,"May 24, 2017",No,No,RX,TARO PHARMACEUTICAL INDUSTRIES LTD


We can see that not all the columns are necessary. Applicant, Strengthm App_No, TE_Code, Applicant_Full_Name are not necessary for ou analysis so we remove them, but first we are going to apply some function to remove the salts from the name of the ingredients.

Creating the list of salts:

### Creating a list of salts and removing the salts from the ingredient

In [4]:
list_of_salts = ["bromide", "hydrobromide", "trisodium", "disodium", "sodium", "chloride","hydrochloride", 
                 "dihydrochloride",  "acetate", "phosphate", "diphosphate", "triphosphate", "monophosphate", "hexahydrate", 
                 "succinate", "mesylate", "dimesylate", "tosylate", "fumarate", "oxalate", "besilate", "mepesuccinate", "malate",
                "sulfate", "citrate", "bisulfate", "difumarate", "maleate", "benzoate",
                "calcium", "tartrate", "nitrate"]

Creating a function that removes the salts by applying ' '.join in x.split() which each x is one line from the column "Ingredient"

In [5]:
orange_book["Ingredient"] = orange_book["Ingredient"].str.lower()
orange_book['active_stripped'] = orange_book['Ingredient'].apply(lambda x: ' '.join([word for word in x.split() if word not in list_of_salts]))

### Selecting only columns of interest

In [6]:
orange_book = orange_book[["active_stripped","Ingredient", "DF;Route", "Appl_Type", "Approval_Date", "RLD", "RS"]]

#Now we're going to take a look into the dataset:

print(f'The dataset currently has the shape {orange_book.shape}')

orange_book.head(7)

The dataset currently has the shape (41731, 7)


,active_stripped,Ingredient,DF;Route,Appl_Type,Approval_Date,RLD,RS
0,budesonide,budesonide,"AEROSOL, FOAM;RECTAL",N,"Oct 7, 2014",Yes,Yes
1,minocycline,minocycline hydrochloride,"AEROSOL, FOAM;TOPICAL",N,"Oct 18, 2019",Yes,Yes
2,azelaic acid,azelaic acid,"AEROSOL, FOAM;TOPICAL",A,"Oct 7, 2020",No,No
3,betamethasone valerate,betamethasone valerate,"AEROSOL, FOAM;TOPICAL",A,"Nov 26, 2012",No,No
4,betamethasone valerate,betamethasone valerate,"AEROSOL, FOAM;TOPICAL",A,"May 24, 2017",No,No
5,betamethasone valerate,betamethasone valerate,"AEROSOL, FOAM;TOPICAL",A,"May 24, 2017",No,No
6,clindamycin,clindamycin phosphate,"AEROSOL, FOAM;TOPICAL",A,"Sep 20, 2021",No,No


Now that we've selected only, we are going to transform everything into lowercase applying the function below:

In [7]:
# doing a loop to transform everything into lowercase

# maybe there is a better solution to this? maybe a janitor-like library
for column in orange_book.columns:
    orange_book[column] = orange_book[column].str.lower()

#### So here are the steps we're going to take next:
- strip route from dosage form
- strip the approval date to only year and select year 1997-2021
- the application type is only valid to N and RS == "yes"
- finally grouping the different routes for each ingredient

Here we use the function to split the column "DF;Route" into two strings using the delimiter ";" and selecting the second string stripped, that is, after the delimiter:

In [8]:
orange_book["Route"] = orange_book["DF;Route"].str.split(";").str[1]

We do the same thing to retrieve the approval year:

In [9]:
orange_book["Approval_Year"] = orange_book["Approval_Date"].str.split(",").str[1]

Performing a series of filters to the dataset:

### Filtering only new drug applications

In [10]:
print(f'Initially the dataset contains {orange_book.shape[0]} lines')

# First we only Appl_Type == "n" to new drug applications
new_drugs = orange_book.loc[orange_book["Appl_Type"] == "n"]

print(f'Now the dataset of only new drug aplications has {new_drugs.shape[0]} lines')

Initially the dataset contains 41731 lines
Now the dataset of only new drug aplications has 9975 lines


### Filtering only RS == "yes"

In [11]:
# We aplly the only RS == "yes" filter

new_drugs = new_drugs.loc[new_drugs["RS"] == "yes"]
print(f'After the filter (RS == "yes") the dataset contains {new_drugs.shape[0]} lines')

After the filter (RS == "yes") the dataset contains 2937 lines


### Filtering only Approval Year 1997 and 2021

In [12]:
new_drugs["Approval_Year"] = new_drugs["Approval_Year"].astype(int)

new_drugs = new_drugs.loc[(new_drugs["Approval_Year"] >= 1997) & (new_drugs["Approval_Year"] <= 2021)]

print(f'After the filter (Approval_Year 1997-2021) the dataset contains {new_drugs.shape[0]} lines')

After the filter (Approval_Year 1997-2021) the dataset contains 2046 lines


Do not forget to reset the index

In [13]:
new_drugs.reset_index(drop = True)

,active_stripped,Ingredient,DF;Route,Appl_Type,Approval_Date,RLD,RS,Route,Approval_Year
0,budesonide,budesonide,"aerosol, foam;rectal",n,"oct 7, 2014",yes,yes,rectal,2014
1,minocycline,minocycline hydrochloride,"aerosol, foam;topical",n,"oct 18, 2019",yes,yes,topical,2019
2,econazole,econazole nitrate,"aerosol, foam;topical",n,"oct 24, 2013",yes,yes,topical,2013
3,betamethasone dipropionate; calcipotriene,betamethasone dipropionate; calcipotriene,"aerosol, foam;topical",n,"oct 16, 2015",yes,yes,topical,2015
4,clindamycin,clindamycin phosphate,"aerosol, foam;topical",n,"oct 22, 2004",yes,yes,topical,2004
...,...,...,...,...,...,...,...,...,...
2041,buprenorphine hydrochloride; naloxone,buprenorphine hydrochloride; naloxone hydrochl...,tablet;sublingual,n,"dec 11, 2014",yes,yes,sublingual,2014
2042,estradiol,estradiol,tablet;vaginal,n,"nov 25, 2009",yes,yes,vaginal,2009
2043,nicotine polacrilex,nicotine polacrilex,troche/lozenge;oral,n,"may 18, 2009",yes,yes,oral,2009
2044,nicotine polacrilex,nicotine polacrilex,troche/lozenge;oral,n,"oct 31, 2002",yes,yes,oral,2002


We also need to remove every active_stripped that has an ";" because it is an association and we're not going to need that

In [14]:
# we performing another filtering
new_drugs = new_drugs[~new_drugs.active_stripped.str.contains(r';')].reset_index(drop = True)

# Removing the radioactives
new_drugs = new_drugs[~new_drugs.active_stripped.str.contains(r'-?\d+')].reset_index(drop = True)

print(f'{new_drugs.shape[0]} left after the removing of the associations')

new_drugs

1685 left after the removing of the associations


,active_stripped,Ingredient,DF;Route,Appl_Type,Approval_Date,RLD,RS,Route,Approval_Year
0,budesonide,budesonide,"aerosol, foam;rectal",n,"oct 7, 2014",yes,yes,rectal,2014
1,minocycline,minocycline hydrochloride,"aerosol, foam;topical",n,"oct 18, 2019",yes,yes,topical,2019
2,econazole,econazole nitrate,"aerosol, foam;topical",n,"oct 24, 2013",yes,yes,topical,2013
3,clindamycin,clindamycin phosphate,"aerosol, foam;topical",n,"oct 22, 2004",yes,yes,topical,2004
4,ketoconazole,ketoconazole,"aerosol, foam;topical",n,"jun 12, 2007",yes,yes,topical,2007
...,...,...,...,...,...,...,...,...,...
1680,asenapine,asenapine maleate,tablet;sublingual,n,"aug 13, 2009",yes,yes,sublingual,2009
1681,estradiol,estradiol,tablet;vaginal,n,"nov 25, 2009",yes,yes,vaginal,2009
1682,nicotine polacrilex,nicotine polacrilex,troche/lozenge;oral,n,"may 18, 2009",yes,yes,oral,2009
1683,nicotine polacrilex,nicotine polacrilex,troche/lozenge;oral,n,"oct 31, 2002",yes,yes,oral,2002


We can take another look at the dataset

In [15]:
print(f'There are {len(new_drugs["active_stripped"].unique())} are unique actives, se we need to remove more or group_by')

There are 857 are unique actives, se we need to remove more or group_by


Now we need to group_by active_stripped and join the different routes into the same line, the solution is found here:
solution found here: https://www.skytowner.com/explore/merging_rows_within_a_group_together_in_pandas

In [16]:
actives_route = new_drugs.groupby("active_stripped", as_index = False).agg({"active_stripped":"first", "Route": ' '.join})
actives_route

,active_stripped,Route
0,,oral injection injection intravenous oral
1,abacavir,oral oral
2,abaloparatide,subcutaneous
3,abemaciclib,oral
4,abiraterone,oral oral
...,...,...
852,zirconium cyclosilicate,oral
853,zoledronic acid,intravenous intravenous intravenous
854,zolmitriptan,nasal oral oral
855,zolpidem,oral oral sublingual


But now we need to remove duplicated route strings. We can do this by applying a <b>lambda function</b> coupled with a <b>set</b> in each line:

In [17]:
actives_route['Route_Unique'] = actives_route['Route'].apply(lambda x: ' '.join([word for word in set(x.split())]))
actives_route = actives_route[["active_stripped", "Route_Unique"]]

In [18]:
actives_route = actives_route.drop([0]).reset_index(drop = True)

Apparently everything is okay, now lets save it and explore it a little further

In [19]:
actives_route.to_csv("../data/manually_curated_datasets/active_route_final.csv")